In [171]:
w = "\n\nblah\n\n"
prob_types, prob_counts = np.unique(["1","2","1"],return_counts=True)
w += "The fitter threw the following:\n"
for i in range(len(prob_types)):
    w += f"  '{prob_types[i]}', {prob_counts[i]} times.\n"

import warnings
warnings.warn(w)

/var/folders/p_/hcs03cdj48nbsvw72d92tr040000gn/T/ipykernel_52576/950956994.py:8: UserWarning: 

blah

The fitter threw the following:
  '1', 2 times.
  '2', 1 times.

  warnings.warn(w)


In [180]:
help(scipy.optimize.least_squares)


Help on function least_squares in module scipy.optimize._lsq.least_squares:

least_squares(fun, x0, jac='2-point', bounds=(-inf, inf), method='trf', ftol=1e-08, xtol=1e-08, gtol=1e-08, x_scale=1.0, loss='linear', f_scale=1.0, diff_step=None, tr_solver=None, tr_options={}, jac_sparsity=None, max_nfev=None, verbose=0, args=(), kwargs={})
    Solve a nonlinear least-squares problem with bounds on the variables.

    Given the residuals f(x) (an m-D real function of n real
    variables) and the loss function rho(s) (a scalar function), `least_squares`
    finds a local minimum of the cost function F(x)::

        minimize F(x) = 0.5 * sum(rho(f_i(x)**2), i = 0, ..., m - 1)
        subject to lb <= x <= ub

    The purpose of the loss function rho(s) is to reduce the influence of
    outliers on the solution.

    Parameters
    ----------
    fun : callable
        Function which computes the vector of residuals, with the signature
        ``fun(x, *args, **kwargs)``, i.e., the minimizati

In [177]:
e

NameError: name 'e' is not defined

In [167]:
import scipy

def yo(a): return np.nan*np.ones(10)

scipy.optimize.least_squares(fun=yo,
                             x0=np.ones(2,dtype=float),
                             bounds=np.array([[-np.inf,-np.inf],[np.inf,np.inf]]))

ValueError: Residuals are not finite in the initial point.

In [153]:
%matplotlib inline
import emcee
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [6]:
def ln_prob(param): return -param[0]*param[1]
num_steps = 10
num_walkers = 10
ndim = 2
pos = [np.random.normal(loc=0,scale=1,size=2)
       for _ in range(num_walkers)]

es = emcee.EnsembleSampler(num_walkers,
                           ndim,
                           ln_prob)
yo = es.run_mcmc(pos,num_steps)

In [151]:
help(emcee.EnsembleSampler)

Help on class EnsembleSampler in module emcee.ensemble:

class EnsembleSampler(builtins.object)
 |  EnsembleSampler(nwalkers, ndim, log_prob_fn, pool=None, moves=None, args=None, kwargs=None, backend=None, vectorize=False, blobs_dtype=None, parameter_names: Union[Dict[str, int], List[str], NoneType] = None, a=None, postargs=None, threads=None, live_dangerously=None, runtime_sortingfn=None)
 |
 |  An ensemble MCMC sampler
 |
 |  If you are upgrading from an earlier version of emcee, you might notice
 |  that some arguments are now deprecated. The parameters that control the
 |  proposals have been moved to the :ref:`moves-user` interface (``a`` and
 |  ``live_dangerously``), and the parameters related to parallelization can
 |  now be controlled via the ``pool`` argument (:ref:`parallel`).
 |
 |  Args:
 |      nwalkers (int): The number of walkers in the ensemble.
 |      ndim (int): Number of dimensions in the parameter space.
 |      log_prob_fn (callable): A function that takes a vec

In [61]:
np.random.random(10)

array([0.47628129, 0.96412469, 0.24411467, 0.5481956 , 0.32095011,
       0.52541791, 0.63221125, 0.48021056, 0.46275819, 0.73930419])

In [148]:
def _sample_gaussian(prior_mean,
                     prior_std,
                     lower_bound,
                     upper_bound,
                     num_walkers):
    
    # generate a huge number of possible priors 
    gaussian_priors = np.random.normal(loc=prior_mean,
                                       scale=prior_std,
                                       size=num_walkers*1000)
    
    # Grab only those priors that are within the bounds
    good_mask = np.logical_and(gaussian_priors > lower_bound,
                               gaussian_priors < upper_bound)
    good_priors = gaussian_priors[good_mask]
    
    # If we have enough good priors, keep them. If we have only a few
    # good priors, it means the bounds have sliced out a ridiculously
    # tiny chunk of the distribution. Approximate the walkers as 
    # a uniform sample from that distribution. 
    if len(good_priors) >= num_walkers:
        return good_priors[:num_walkers]

    return None

def _sample_uniform(lower_bound,
                    upper_bound,
                    num_walkers,
                    infinity_proxy):

    # Slice down infinite bounds to largish numbers
    if np.isinf(lower_bound): 
        lower_bound = -infinity_proxy
    if np.isinf(upper_bound):
        upper_bound = infinity_proxy

    # If only one walker, put at the mean of the bounds
    if num_walkers == 1:
        return [np.mean([lower_bound,upper_bound])]

    # If the upper and lower bounds have the same sign, make a uniform
    # span between them (log steps). For example, 1e-9 to 1e-6 with four
    # walkers would yield 1e-9, 1e-8, 1e-7, 1e-6
    if upper_bound*lower_bound > 0:
        
        steps = np.exp(np.arange(num_walkers))
        steps = (steps - np.min(steps))/(np.max(steps) - np.min(steps))
        walkers = steps*(upper_bound - lower_bound) + lower_bound
        np.random.shuffle(walkers)
        
        return walkers

    # If the upper and lower bounds have different signs, make uniform
    # spans from 0 to upper and 0 to lower, weighted by how much of the
    # the span is above and below. 
    
    # Figure out fraction of uniform distribution below zero
    lower_mag = np.abs(lower_bound)
    upper_mag = np.abs(upper_bound)
    fx_lower = lower_mag/(lower_mag + upper_mag)

    # Figure out how many walkers to place above and below zero
    num_below = int(np.round(fx_lower*num_walkers,0))

    # Make sure we have at least one above and one below
    if num_below == 0: 
        num_below = 1
    if num_below == num_walkers:
        num_below = num_walkers - 1
    num_above = num_walkers - num_below

    # Create steps from 0 to upper_bound
    steps = np.exp(np.arange(num_above))
    steps = (steps - np.min(steps))/(np.max(steps) - np.min(steps))
    above_walkers = list(steps*upper_bound)

    # Create steps from 0 to lower_bound
    steps = np.exp(np.arange(num_below))
    steps = (steps - np.min(steps))/(np.max(steps) - np.min(steps))
    below_walkers = list(steps*lower_bound)

    # Combine all steps
    above_walkers.extend(below_walkers)
    walkers = np.array(above_walkers)

    # Shuffle randomly
    np.random.shuffle(walkers)

    return walkers


def _create_walkers(param_df,
                    num_walkers,
                    infinity_proxy=1e9):

    walker_list = []

    # Go through each parameter one-by-one
    for p in param_df.index:
        
        # Skip fixed parameters
        if param_df.loc[p,"fixed"]:
            continue

        # Get prior mean, std, and bounds
        guess = param_df.loc[p,"guess"]
        prior_mean = param_df.loc[p,"prior_mean"]
        prior_std = param_df.loc[p,"prior_std"]
        lower_bound = param_df.loc[p,"lower_bound"]
        upper_bound = param_df.loc[p,"upper_bound"]

        # If gaussian prior, try to do that first. 
        if not np.isnan(prior_mean):

            gaussian_priors = _sample_gaussian(prior_mean,
                                               prior_std,
                                               lower_bound,
                                               upper_bound,
                                               num_walkers)
            if gaussian_priors is not None:
                walker_list.append(gaussian_priors)
                continue

        # If we get here, gaussian priors were not given or did not work.
        uniform_priors = _sample_uniform(lower_bound,
                                         upper_bound,
                                         num_walkers,
                                         infinity_proxy)
        walker_list.append(uniform_priors)
        
    walkers = np.array(walker_list).T

    return walkers


param_df = pd.DataFrame({"name":["a","b"],
                        "guess":[1,2],
                        "fixed":[False,False],
                        "prior_mean":[np.nan,np.nan],
                        "prior_std":[2,np.nan],
                        "lower_bound":[-0.001,-np.inf],
                        "upper_bound":[0.001,np.inf]})

s = _create_walkers(param_df=param_df,
                num_walkers=10)
s

array([[-1.19202922e-04,  3.20586033e+07],
       [-0.00000000e+00,  1.19202922e+08],
       [ 0.00000000e+00,  3.56085740e+08],
       [-3.20586033e-05, -1.19202922e+08],
       [-3.56085740e-04,  0.00000000e+00],
       [ 1.00000000e-03, -3.20586033e+07],
       [-1.00000000e-03, -3.56085740e+08],
       [ 3.20586033e-05, -1.00000000e+09],
       [ 3.56085740e-04, -0.00000000e+00],
       [ 1.19202922e-04,  1.00000000e+09]])

In [131]:
np.log(10)
np.log(0.001)

-6.907755278982137

In [46]:
prior_dist = stats.sampling.FastGeneratorInversion(rv)
x = prior_dist.rvs((10,2))
means = np.array([0,1])
stds = np.array([10,1])

In [50]:
(x - means)/stds

array([[-0.00309253, -0.65680484],
       [-0.10013265, -2.06772866],
       [-0.01153172, -0.97799837],
       [ 0.29657577, -0.38081001],
       [-0.06001777,  0.47087776],
       [-0.03209133, -2.95918773],
       [ 0.06411045, -1.43260952],
       [-0.09383089, -0.040393  ],
       [-0.03906135, -1.07416001],
       [ 0.06609788, -1.31870454]])